# Poem Generator

In this notebook we build upon the web scraping to create a RNN that can generate Arabic poems in the style of Nizar Qabbani.

In [1]:
import glob
import re
import numpy as np
from collections import Counter
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import helper

In [2]:
#get list of text files in data
poem_txt_list = glob.glob('data/*.txt')

In [3]:
with open('raw_corpus.txt', 'w') as outfile:
    for fname in poem_txt_list:
        with open(fname) as infile:
            outfile.write(infile.read())

In [4]:
data_dir = 'raw_corpus.txt'
text = helper.load_data(data_dir)

In [5]:
view_line_range = (0, 10)

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 31980
Number of lines: 24304
Average number of words in each line: 3.9885615536537196

The lines 0 to 10:
تحركي خطوةً.. يا نصف عاشقةٍ 
فلا أريد أنا أنصاف عشاقٍ 
إن الزلازل طول الليل تضربني 
وأنت واضعةٌ ساقاً على ساق 
وأنت آخر من تعنيه مشكلتي 
ومن يشاركني حزني وإرهاقي 
تبللي مرةً بالماء .. أو بدمي 
وجربي الموت يوماً فوق أحداقي 
أنا غريبٌ.. ومنفيٌ.. ومستلب 
وثلج نهديك غطى كل أعماقي 


---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  We implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, we first need to transform the words to ids.  In this function, we create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

We return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [6]:
def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    
    #create a counter for all words in text
    word_counts = Counter(text)
    
    #sort words from most to least frequent in the text
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    
    #create int to vocab dictionatires
    
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word:ii for ii, word in int_to_vocab.items()}
    
    # return tuple
    return (vocab_to_int, int_to_vocab)

### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

We implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||". We create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Ensure we don't use a value that could be confused as a word; for example, instead of using the value "dash", try something like "||dash||".

**Note that we might need to copy from the text for some punctuation such as the question mark which has a fliped orientation in arabic script**

In [7]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    punct_dict = {'.': '||PERIOD||',  
                  '*': '||COMMA||', 
                  '\r': '||RECUR||', 
                  '…': '||DOTDOTDOT||', 
                  '!': '||EXCLAMATIONMARK||', 
                  '؟ ': '||QUESTIONMARK||',  
                  '(': '||LEFTPARANTH||',
                  ')': '||RIGHTPARANTH||', 
                  '–': '||DASH||', 
                  '\n': '||RETURN||'}
        
    return punct_dict

## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. Please check out at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail.

In [8]:
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)
int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, we build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [9]:
# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

No GPU found. Please use a GPU to train your neural network.


## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

We can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
We implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>We can batch words using the DataLoader, but it will be up to us to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Our first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [10]:
def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    
    #number of batches by integer definition
    n_batches = len(words)//batch_size
    
    #only full batches
    words = words[:n_batches*batch_size]
    y_len = len(words) - sequence_length
    
    x, y = [], []
    for idx in range(0, y_len):
        end = idx + sequence_length
        x_batch = words[idx:end]
        y_batch = words[end]
        
        x.append(x_batch)
        y.append(y_batch)
    
    #wrapping tensor
    data = TensorDataset(torch.from_numpy(np.asarray(x)), torch.from_numpy(np.asarray(y)))
    # Combines a dataset and a sampler, and provides single- or multi-process iterators over the dataset
    data_loader = DataLoader(data, batch_size=batch_size)
    
    # return a dataloader
    return data_loader


### Test The Dataloader 

We have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Our code should return something like the following (likely in a different order, if we shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Our sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

We should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [11]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 0,  1,  2,  3,  4],
        [ 1,  2,  3,  4,  5],
        [ 2,  3,  4,  5,  6],
        [ 3,  4,  5,  6,  7],
        [ 4,  5,  6,  7,  8],
        [ 5,  6,  7,  8,  9],
        [ 6,  7,  8,  9, 10],
        [ 7,  8,  9, 10, 11],
        [ 8,  9, 10, 11, 12],
        [ 9, 10, 11, 12, 13]])

torch.Size([10])
tensor([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14])


---
## Build the Neural Network
We implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). We may choose to use a GRU or an LSTM. To complete the RNN, we have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Notes

1. Ensure to stack the outputs of the lstm to pass to our fully-connected layer, we can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. We can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [12]:
class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5, lr=0.001):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        #Implement function
        
        #set class variables
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.output_size = output_size
        
        # embedding layer
        self.embedding = nn.Embedding(num_embeddings = vocab_size, embedding_dim = embedding_dim)
        
        # define lstm
        self.lstm = nn.LSTM(input_size = embedding_dim, 
                            hidden_size = hidden_dim, 
                            num_layers = n_layers,
                            bias = True, 
                            batch_first = True,
                            dropout = dropout)
           
        #define fc layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        
        #get the batch size
        batch_size = nn_input.size(0)
        
        #get embedding
        embed = self.embedding(nn_input)
        
        #get lstm output
        out, hidden = self.lstm(embed, hidden)
        
        #stack the outputs of the lstm
        out = out.contiguous().view(-1, self.hidden_dim)        
        
        out = self.fc(out)
        
        
        # reshape into (batch_size, seq_length, output_size)
        out = out.view(batch_size, -1, self.output_size)
        # get last batch
        out = out[:, -1]
        
        # return one batch of output word scores and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        
        # initialize hidden state with zero weights, and move to GPU if available
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

### Define forward and backpropagation

Use the RNN class we implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that we can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available,  move your data to that GPU device, here.**

In [13]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden, clip=5):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :param clip: Max norm of the gradients
    :return: The loss and the latest hidden state Tensor
    """
        
    # move data to GPU, if available
    if train_on_gpu:
        rnn.cuda()
    
    # perform backpropagation and optimization
    

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in hidden])

    # zero accumulated gradients
    rnn.zero_grad()
            
    # get the output from the model
    if train_on_gpu:
        inp = inp.cuda()
        target = target.cuda()
    
    
    output, h = rnn(inp, h)
            
    # calculate the loss and perform backprop
    loss = criterion(output, target)
    loss.backward()
    
    # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
    nn.utils.clip_grad_norm_(rnn.parameters(), clip)
    optimizer.step()
    

    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), h

## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. We set this parameter along with other parameters in the next section.

In [14]:
def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

We set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of our RNN.
- Set `n_layers` to the number of layers/cells in our RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [15]:
# Data params
# Sequence Length = # of words in a sequence
sequence_length =  10 
# Batch Size
batch_size = 128

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [16]:
# Training parameters
# Number of Epochs
num_epochs = 15
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 250
# Hidden Dimension
hidden_dim = 512
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 500

### Train
In the next cell, we train the neural network on the pre-processed data.  If we have a hard time getting a good loss, we may consider changing the hyperparameters. In general, we may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **Note: aim for a loss less than 3.5.** 

We can  also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [17]:
# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 15 epoch(s)...
Epoch:    1/15    Loss: 6.76675343799591

Epoch:    1/15    Loss: 6.370319797515869

Epoch:    2/15    Loss: 5.788004631557256

Epoch:    2/15    Loss: 5.498991756439209

Epoch:    3/15    Loss: 5.315837315389779

Epoch:    3/15    Loss: 5.086999155521393

Epoch:    4/15    Loss: 4.981034511894964

Epoch:    4/15    Loss: 4.744471074104309

Epoch:    5/15    Loss: 4.643881075653755

Epoch:    5/15    Loss: 4.411609432220459

Epoch:    6/15    Loss: 4.350242452576826

Epoch:    6/15    Loss: 4.088176232337951

Epoch:    7/15    Loss: 4.028933911911225

Epoch:    7/15    Loss: 3.7675121421813964

Epoch:    8/15    Loss: 3.6930512214971594

Epoch:    8/15    Loss: 3.4714884057044983

Epoch:    9/15    Loss: 3.3951571801802296

Epoch:    9/15    Loss: 3.1781059091091155

Epoch:   10/15    Loss: 3.096065091268507

Epoch:   10/15    Loss: 2.916186452627182

Epoch:   11/15    Loss: 2.8425318199600706

Epoch:   11/15    Loss: 2.6659098613262175

Epoch:   12/15    Lo

/Users/nadimkawwa/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


---
# Checkpoint

After running the above training cell, our model will be saved by name, `trained_rnn`, and if we save our notebook progress, **we can pause here and come back to this code at another time**. We can resume our progress by running the next cell, which will load in our word:id dictionaries _and_ load in our saved model by name.

In [18]:
_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('trained_rnn')

## Generate Poems
With the network trained and saved, we can use it to generate a new Kabbani poem in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. We use the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!


### How to Write Arabic Characters
Some users might have a built in arabic keyboard on their computers. However that might not be the case, so feel free to use smart keyboards such as [yamli.com](https://www.yamli.com/).

In [19]:
def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a Poem
It's time to generate the text. We set `gen_length` to the length of poem we want to generate and set `prime_word` to one of the following to start the prediction:
- "أنا" (I/me)
- "يا" (O)
- "نحن" (We)
- "امرأة" (Woman)

We can set the prime word to _any word_ in our dictionary, but it's best to start with a word that typically begins a sentence in Arabic, or even better: <br>
**Read the poems and try to begin like the original author**

In [21]:
# run the cell multiple times to get different results!
gen_length = 50 # modify the length to your preference
# name for starting the script
prime_word =  'أنا'


pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, 
                            vocab_to_int[prime_word], 
                            int_to_vocab, 
                            token_dict, 
                            vocab_to_int[pad_word], 
                            gen_length)
print(generated_script)

أنا كهذا الماء ؟.
وظلت شيخ حارتنا..
وتشاهد أسماكٌ وبحار.
وتشاهد فهم أنفسنا
الخائفون من الخشب
ويضاجعون تطرف يعنيها.
تنقرض الأمة من الأصل ذكورا..
تهاجر أسماكٌ وبحار..
12
وعدت..
بإلغاء


In [26]:
prime_list = ["أنا",  "يا",  "نحن", "امرأة"]

for word in prime_list:
    print("Generating poem for {}".format(word))
    generated_script = generate(trained_rnn, 
                            vocab_to_int[word], 
                            int_to_vocab, 
                            token_dict, 
                            vocab_to_int[pad_word], 
                            gen_length)
    print(generated_script)
    print(20*'-')
    

Generating poem for أنا
أنا كهذا عصرنا– الحداد
من هؤلاء الطارئون والصندل


يا شـام، حـيٍ.. ومنفيٌ..
وبنثرهم نبيذٍ.. أو تهربي..
وبدأنا أشجار حارتنا
قطعاً أنـت.. وبين الفار..
ووجدنا أشباه مغتصباً..
وتشاهد
--------------------
Generating poem for يا
يا قطتي الخليفة والقصور..
ونحن, في غباء دفتري..
وهم أطلع من أرنبتي نعال وبين ورود.
وبدأنا ننسى أنفسنا جاءها معاويةٍ

لكنه غضـبٌ طـالت أظـافـره
ماذا سيحدث يتغزلون؟ 
كيف أحوالك ، مفرد ، والشذا خطاك؟ 



--------------------
Generating poem for نحن
نحن رحيل الأميره..
وهم أطلع في الأصل إناثاً.
وبدأنا ننسى متجولون ؟؟ 
4
البائعون ثقافةً مغشوشةً
والراقدون بغرفة الإنعاش..
عيونها..
ضفيرةً..
وقصة رصاصهم..
ماذا سيحدث للكواكب؟  وأنت خلتها كـذبا
--------------------
Generating poem for امرأة
امرأة كلها...
9
يا كل عامٍ في الطبيعة..
ومذيع الدولة في جرحنا
نتفاءل جميله..
ووجدنا جسداً مغتصباً..
ومذيع الدولة ؟؟ 
من هؤلاء هؤلاء الهدبـا
من هؤلاء سقيت أعماقي وإرهاقي برأس أدبي؟


--------------------
